# Exercise 1: Trading insights from a fixed list of companies
In this exercise, you will get actionable insights for investment based on recent trading results of the companies you are investing in.

(1) You will first retrieve a list of ticker symbols for companies, then

(2) retrieve the trading results for these companies from Google, then

(3) define a function using the OpenAI API to generate a report of the results and

(4) define a function to generate actionable insights based on the report, and finally

(5) combine the two functions into a sequential chain.

To accomplish this, you can expand the code provided below:

###Setup
First, make the necessary imports.

In [1]:
!pip install -q langchain langchain-community langchain-core langchain-openai

from google.colab import userdata
from transformers import pipeline
from langchain.chains import SequentialChain, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.document_loaders import CSVLoader
from langchain_openai import ChatOpenAI
import ast
import os
import pprint
import getpass
from IPython.display import Markdown

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


Then, assign the API keys to be able to use Google Serper and OpenAI.

When working with sensitive information like API keys or passwords in Google Colab, it's crucial to handle data securely. As you learnt in the tutorial session, two common approaches for this are using **Colab's Secrets Manager**, which stores and retrieves secrets without exposing them in the notebook, and `getpass`, a Python function that securely prompts users to input secrets during runtime without showing them. Both methods help ensure your sensitive data remains protected.

In [2]:
#When using Colab Secret Manager
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')
#When using getpass
#os.environ['SERPER_API_KEY'] = getpass.getpass()

#When using Colab Secret Manager
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
#When using getpass
#os.environ['OPENAI_API_KEY'] = getpass.getpass()

Lastly, define the search model and the llm.

In [3]:
search = GoogleSerperAPIWrapper()

llm = ChatOpenAI(temperature=0.9, model="gpt-4o")

### Step 1. Load the ticker symbols of the companies you invest in
In the first step, you load a list of ticker symbols of companies in the stock market.

Here, we assume that the list is predefined with the symbols of Apple, Amazon, Microsoft, and NVIDIA.

In [4]:
ticker_symbols = ["AAPL", "AMZN", "MSFT", "NVDA"]

### Step 2. Retrieve recent trading results for the ticker symbols

Use the `GoogleSerperAPIWrapper` (which you assigned to `search` in the Setup phase) to run a query for each ticker symbol in `ticker_symbols`.

**Hint:** To do so, iterate over the individual ticker symbols in the list using the "`for ... in ...:`" structure. In each iteration, it is sufficient to just run the search passing the ticker symbol as parameter: the query will return the recent trading results for the respective company. For example, if you search `AAPL`, the result will look like `233.28 -1.12 (0.48%)`.

The result of running the queries (one per ticker symbol) should be a concatenation of the individual results in a single string `trading_results`. The structure of this string should be as follows: a ticker symbol, followed by the respective trading result, and then a semicolon (";"), followed by the same structure for the remaining symbols. An example for the result is as follows:

`AAPL 233.28 -1.12 (0.48%);AMZN 217.76 -0.70 (0.32%);MSFT 415.67 -1.52 (0.36%);NVDA 131.76 -1.47 (1.10%);NVDA 131.76 -1.47 (1.10%);`

In [5]:
# Initialize an empty string where to append the results
trading_results = ""

# Iterate over the ticker symbols to run the query and append the results as specified
## YOUR CODE (START)
for ticker_symbol in ticker_symbols:
    result = search.run(ticker_symbol)
    trading_results = trading_results + ticker_symbol + " " + result + "; "
## YOUR CODE (END)

In [6]:
# Display the trading results in Markdown
display(Markdown(trading_results))

AAPL 232.80 +4.79 (2.10%); AMZN 242.06 +4.64 (1.95%); MSFT 412.37 +1.45 (0.35%); NVDA 118.65 +1.99 (1.71%); 

### Step 3. Create a prompt template and chain to get a report in natural language for the trading results

Use `PromptTemplate` to define a prompt template `report_prompt_template`.

The prompt template should take as input variable the string `trading_results` that was generated in the previous step.

The template should ask the llm to provide a report for the trading results that are in the input variable.

The template should specify that the report must only talk about variations in percent, not in dollars.

The template should also specify that the report must have a maximum length, for example of 200 words.

In [7]:
# Define the prompt template
## YOUR CODE (START)
report_prompt_template = PromptTemplate(
    input_variable =["trading_results"],
    template = ("Based on the a list with ticker symbols {trading_results} create a report with a maximum length of 200 words. The report must only talk about variations in percent, not in dollars.")
)
## YOUR CODE (END)

Define an `LLMChain` called `report_chain` with:
* the `llm` defined in the setup phase as the large language model,
*`report_prompt_template` as the prompt,
* an output called `textual_report`.

In [8]:
# Define the llm chain
## YOUR CODE (START)
report_chain = LLMChain(llm=llm, prompt=report_prompt_template, output_key="textual_report")
## YOUR CODE (END)

<ipython-input-8-5d8badc15cf0>:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  report_chain = LLMChain(llm=llm, prompt=report_prompt_template, output_key="textual_report")


### Step 4. Create a prompt template and chain to generate actionable insights for investors based on the report

Use PromptTemplate to define a prompt template `insight_prompt_template`.

The prompt template should take as input variable `textual_report`, which is the output of the chain defined in the previous step.

The template should ask the llm to provide actionable insights for investors based on the report in the input variable.

The template should also specify that the insights for each company mentioned in the report must have a maximum length, e.g., 50 words.

In [9]:
# Define the prompt template
## YOUR CODE (START)
insight_prompt_template = PromptTemplate(
    input_variables=["textual_report"],
    template=(
        "Based on the following trading report about selected companies, for each mentioned company extract 50-words actionable insights for investors: {textual_report}."
    )
)
## YOUR CODE (END)

Define an LLMChain called `report_chain` with:
* the `llm` defined in the setup phase as the large language model,
* `insight_prompt_template` as the prompt,
* an output called `insights`.

In [10]:
# Define the llm chain
## YOUR CODE (START)
insight_chain = LLMChain(llm=llm, prompt=insight_prompt_template, output_key="insights")
## YOUR CODE (END)

### Step 5. Create a sequential chain where the trading results from Step 2 are used as input and the daily report and actionable insights are printed

Use `SequentialChain` to define a chain called `complete_chain`, with:
* `report_chain` and `insight_chain` as the chains,
* `trading_results` as the input variable,
* `textual_report` and `insights` as the output variables.

In [11]:
# Combine the chains
## YOUR CODE (START)
complete_chain = SequentialChain(
    chains=[report_chain, insight_chain],
    input_variables=["trading_results"],
    output_variables=["textual_report", "insights"]
)
## YOUR CODE (END)

Invoke the `complete_chain`, passing `trading_results` from Step 2 as the input variable `trading_results`.

The result of the complete chain invocation should be assigned to a variable called `final_response`.

In [12]:
# Run the complete chain
## YOUR CODE (START)
final_response = complete_chain.invoke({"trading_results": trading_results})
## YOUR CODE (END)

Lastly, display the results of the chain invocation to read the report and insights:

In [13]:
# Display the result of the chain
display(Markdown(final_response["textual_report"]))
display(Markdown(final_response["insights"]))

The recent stock performance indicates notable percentage gains across several major technology companies. AAPL (Apple Inc.) experienced a significant increase of 2.10%, marking a robust upward movement in the market. This reflects strong investor confidence and potential positive outlooks in Apple's business operations or market announcements.

Similarly, AMZN (Amazon.com Inc.) reported a rise of 1.95%. Although slightly less than Apple, this increase still highlights a favorable trend in Amazon’s market performance, suggesting that the company is maintaining its competitive edge and appealing to investors.

NVDA (NVIDIA Corporation) also showed a noteworthy rise with a gain of 1.71%. This increase suggests continued optimism from investors regarding NVIDIA's position in the tech industry, possibly driven by advancements in AI and semiconductor technologies.

On the other hand, MSFT (Microsoft Corporation) posted a modest gain of 0.35%. While less pronounced than its peers, this percentage change still points to stability and steady growth, which is characteristic of Microsoft’s market presence.

Overall, these companies reflect positive market sentiment, with each demonstrating varying levels of investor confidence and market performance, likely influenced by their respective business strategies and external market conditions.

1. **Apple Inc. (AAPL)**: The 2.10% increase in Apple's stock suggests strong investor confidence. Investors might consider this an opportunity to capitalize on anticipated positive business developments or announcements. Monitoring future announcements from Apple could provide further insights into potential growth catalysts and validate the current bullish sentiment.

2. **Amazon.com Inc. (AMZN)**: With a 1.95% rise, Amazon maintains its competitive edge, signaling healthy market performance. Investors should consider this an indicator of Amazon's continued appeal. Keeping an eye on Amazon’s strategic initiatives and market expansions can offer additional investment opportunities amidst this positive trend.

3. **NVIDIA Corporation (NVDA)**: NVIDIA's 1.71% gain reflects optimism around its advancements in AI and semiconductors. Investors should consider the potential growth in these sectors as a lucrative opportunity. Keeping track of NVIDIA’s innovations and market positioning can provide insights into sustained investment value.

4. **Microsoft Corporation (MSFT)**: Despite a modest 0.35% gain, Microsoft's stability may appeal to risk-averse investors seeking steady growth. This suggests confidence in Microsoft’s market presence. Investors might focus on Microsoft’s strategic initiatives in cloud computing and enterprise solutions for long-term value.

# Exercise 2: Testing with different lists of companies

In exercise 1, you used a list of given ticker symbols, set in Step 1 with `ticker_symbols = ["AAPL", "AMZN", "MSFT", "NVDA"]`.

Test the code with different ticker symbols. For this, modify the list `ticker_symbols` by replacing `["AAPL", "AMZN", "MSFT", "NVDA"]` with other ticker symbols.

Re-execute Step 2 and Step 5 from exercise 1, this time using a new list of four ticker symbols of your choice. There is no need to re-execute Step 3 and Step 4 as the prompt templates and chains can be reused.

**Hint:** You can refer to https://stockanalysis.com/stocks/ to get inspiration and find new symbols. You can modify the list at the beginning of the exercise, without having to write new code here.

# Exercise 3: Testing with different models

In exercise 1, you initially defined the LLM in the Setup phase with `llm = ChatOpenAI(temperature=0.9, model="gpt-4o")`, and used this LLM throughout all subsequent steps.

In this exercise, you will test the chain with different models. For this, you can just replace `gpt-4o` with another model of your choice, such as `gpt-4o-mini`, `gpt-4-turbo`, `gpt-3.5-turbo`.

Then re-execute Steps 3-5 of your solution to exercise 1, this time with the redefined `llm`, using a different model of your choice for each step.

**Hint:** You can refer to https://platform.openai.com/docs/models for a list of available models. You can modify the code cells in exercise 1, without having to write new code here.

# Exercise 4: Trading insights from a CSV file with companies

In the previous exercises, you used a fixed lists of ticker symbols, which is not flexible.

In this exercise, you will extend the solution of exercise 1, so that the list of ticker symbols is loaded from a CSV file `ticker_symbols.csv`.

To accomplish this, you can first expand the code provided below. More specifically, expand the code in Step 1; for Steps 2--5, you can then re-execute the code cells defined above (exercise 1), since the prompt templates and the sequential chain are reusable.

### Step 1: Load from a CSV file the ticker symbols of the companies you invest in

In the first step, you load a list of ticker symbols of companies in the stock market. The list is provided in a CSV file `ticker_symbols.csv`. You can find the CSV File in Canvas. Store the file locally on your computer and upload it using the upload option at the top of the file-explorer pane to upload any file(s) from your local file system to Colab in the present working directory.

Use `CSVLoader` to load the CSV file into a variable called `ticker_document`.

In [14]:
# Load and split the CSV content
## YOUR CODE (START)
loader = CSVLoader(file_path="/content/ticker_symbols.csv")
ticker_document = loader.load()
## YOUR CODE (END)

Now that the CSV file is loaded into `ticker_document`, you can ask an LLM to read `ticker_document` and return a list of the ticker symbols in it.

To do so, use `PrompTemplate` to define a prompt template called `ticker_prompt_template`.

The prompt template should take an input variable `document`.

The template should ask to provide a list with the ticker symbols extracted from the document in input.

In [15]:
# Define the prompt template
## YOUR CODE (START)
ticker_prompt_template = PromptTemplate(
    input_variables = ["document"],
    template = "Give me a list with the ticker symbols extracted from the attached document: {document}. The response should just be a list of strings in double quotes"
)
## YOUR CODE (END)

Next, define an LLMChain called `ticker_chain` with:
* the `llm` defined in the setup phase as the large language model,
* `ticker_prompt_template` as the prompt,
* an output called `ticker_symbols`.

In [16]:
# Define the llm chain
## YOUR CODE (START)
ticker_chain = LLMChain(llm=llm, prompt=ticker_prompt_template, output_key="ticker_symbols")
## YOUR CODE (END)

Now run `ticker_chain` passing `ticker_document` as parameter; the output of the chain should be assigned to variable `ticker_symbols_from_llm`. Note that the output of the chain is a string.

In [17]:
#Run the chain
## YOUR CODE (START)
ticker_symbols_from_llm = ticker_chain.invoke({"document":ticker_document})
## YOUR CODE (END)

Lastly, convert the string `ticker_symbols_from_llm` into a list called `ticker_symbols`.

To do so, you can use function `ast.literal_eval`, passing `ticket_symbols_from_llm` as parameter.

Assign the result of the invocation to variable `ticker_symbols`, and print it.

In [19]:
#Run the chain
## YOUR CODE (START)
ticker_symbols = ast.literal_eval(ticker_symbols_from_llm["ticker_symbols"])
print(ticker_symbols)
## YOUR CODE (END)

['AAPL', 'AMZN', 'MSFT', 'NVDA']


Now you have a list `ticker_symbols` that originates from a CSV file and that you can use to get trading results and actionable insights, like in exercise 1.

The continuation now is the same as in exercise 1: once you execute the code in the cells of the Setup phase and Step 1 of exercise 4, you can continue by re-executing the code in the cells of Steps 2--5 above.